# Module 07: Complete Image Classification Project

**Build a Production-Ready Classifier**

Put everything together! Build a complete CIFAR-10 classifier from scratch.

## Project Goals
- Load and explore CIFAR-10 dataset
- Implement data augmentation
- Build and train a CNN
- Evaluate with confusion matrix
- Save and load models
- Make predictions on new images

## Time: 60 minutes

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Part 1: CIFAR-10 Dataset

**CIFAR-10:** 60,000 32×32 color images in 10 classes

**Classes:**
0. airplane
1. automobile
2. bird
3. cat
4. deer
5. dog
6. frog
7. horse
8. ship
9. truck

**Split:** 50,000 training + 10,000 testing

In [ ]:
# Load CIFAR-10
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = datasets.CIFAR10("../data/datasets", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10("../data/datasets", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {classes}")

## Part 2: Build the Model

In [ ]:
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 256 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


model = CIFAR10CNN().to(device)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## Part 3: Train the Model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
epochs = 20
history = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

print("Training started...")
for epoch in range(epochs):
    # Train
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    train_acc = 100 * correct / total
    history["train_loss"].append(train_loss / len(train_loader))
    history["train_acc"].append(train_acc)

    # Test
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()

    test_acc = 100 * correct / len(test_loader.dataset)
    history["test_loss"].append(test_loss / len(test_loader))
    history["test_acc"].append(test_acc)

    scheduler.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")

print(f"\nFinal Test Accuracy: {history['test_acc'][-1]:.2f}%")

## Part 4: Evaluation with Confusion Matrix

In [ ]:
# Generate predictions
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(target.numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=classes))

## Part 5: Save and Load Model

In [ ]:
# Save model
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "history": history,
        "classes": classes,
    },
    "../models/saved_models/cifar10_cnn.pth",
)

print("Model saved!")

# Load model
checkpoint = torch.load("../models/saved_models/cifar10_cnn.pth")
model.load_state_dict(checkpoint["model_state_dict"])
print("Model loaded successfully!")

## Summary

You built a complete image classification system!

### What You Accomplished:
- Loaded and preprocessed CIFAR-10
- Built a modern CNN with batch norm and dropout
- Trained for 20 epochs
- Evaluated with confusion matrix
- Saved model for deployment

### Next: Module 08 - Object Detection
Move beyond classification to detecting multiple objects!